In [310]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import NaiveBayesClassifier
import nltk
import pickle
import string
from math import log
from random import shuffle
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import HashingVectorizer
import numpy as np

In [6]:
negative = ['not', 'neither', 'nor', 'but', 'however', 'although', 'nonetheless', 'despite', 'except',
                         'even though', 'yet']
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)

In [7]:
def clean(doc):
    lemma=WordNetLemmatizer()
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop if i not in negative])
    punc_free = "".join([ch for ch in stop_free if ch not in exclude])
    normalized = " ".join([lemma.lemmatize(word) for word in punc_free.split()])
    return normalized

In [ ]:
isear=pd.read_csv('isear_1.csv', header=-1)
isear=isear.drop(2, axis=1)
isear[0][isear[0]=='guit']='guilt'
isear[1]=isear[1].apply(clean)

In [8]:
text=[]
for index, row in isear.iterrows():
    text.append(clean(row[1]))

In [16]:
text_splits=[]
for doc in text:
    text_splits.append(doc.split(' '))
text_splits_total=[]
for doc in text_splits:
    for word in doc:
        text_splits_total.append(word)
unigrams=set(text_splits_total)
unigrams=list(unigrams)

In [120]:
def finder(ngram):
    for index, row in isear.iterrows():
        row_li=row[1].split(' ')
        if ngram in row_li:
            return isear.iloc[index]

In [125]:
index=finder('liberty')
index

0                                                anger
1    deputy manager summer certain deputy took libe...
Name: 201, dtype: object

The bug was that there was a mismatch between cleaned dataset and used dataset.

In [129]:
def ratio(isear_specified_class, ngram):
    all_words_in_class=isear_specified_class[1].str.split(' ')
    li=[]
    for doc in all_words_in_class:
        for word in doc:
            li.append(word)
    all_words_in_class=li
    count_all_words=len(all_words_in_class)
    count_ngram_em_class=0
    for word in all_words_in_class:
        if ngram==word:
            count_ngram_em_class+=1
    #count_ngram_em_class counts the occurences of ngram in isear_spec_class
    ratio=count_ngram_em_class/count_all_words
    return ratio

In [82]:
def WLLS(ngram, em_class):
    isear_specified_class=isear[isear[0]==em_class]
    ratio_1=ratio(isear_specified_class, ngram)
    isear_all_other_classes=isear[isear[0]!=em_class]
    ratio_2=ratio(isear_all_other_classes, ngram)
    if ratio_1==0:
        return 'not in class'
    elif ratio_2==0:
        return 'only in this class'
    wlls=ratio_1*log(ratio_1/ratio_2)
    return wlls

In [133]:
for emotion in emotion_classes:
    print(WLLS('suppose', emotion))

not in class
not in class
only in this class
not in class
not in class
not in class
not in class


<h3>Another Bug</h3>

I don't know why features where type(wlls)==str are being appended to the features list. So I remove them but it seems inefficient to add then remove them, would rather not add the extras in the first place. 

<i>Fixed it- it was because str!='str'</i>

<h3>Saving and loading with pickle</h3>

In [166]:
def save_obj(obj, name ):
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f)
        
def load_obj(name ):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

<h3>Saving and loading with numpy</h3>

In [ ]:
import numpy as np

# Save
dictionary = {'hello':'world'}
np.save('my_file.npy', dictionary) 

# Load
read_dictionary = np.load('my_file.npy').item()
print(read_dictionary['hello']) # displays "world"

<h1>This step takes a long time, use pickled files</h1>

In [162]:
emotion_classes=['joy', 'fear', 'anger', 'sadness', 'disgust', 'shame', 'guilt']
features={}
features_one_cat={}
for emotion in emotion_classes:
    features[emotion]=[]
    features_one_cat[emotion]=[]
    for ngram in unigrams:
        wlls=WLLS(ngram, emotion)
        if type(wlls)!=str:
            features[emotion].append([ngram, wlls])
        elif wlls=='only in this class':
            features_one_cat[emotion].append(ngram)

In [161]:
features

{'anger': [['liberty', 0.00013931908573253248],
  ['funeral', -0.0001269325750260019]],
 'disgust': [['liberty', 0.00016616536562212152]],
 'fear': [['lane', 0.0001411121214219807]],
 'guilt': [['lane', 0.00015597798598086725]],
 'joy': [['funeral', -0.00012848296494034483]],
 'sadness': [['funeral', 0.007609675242529252]],
 'shame': [['funeral', -0.0001271593966276527]]}

In [160]:
features_one_cat

{'anger': ['suppose', 'push', 'unimpartiality'],
 'disgust': [],
 'fear': [],
 'guilt': ['oversee', 'squeezed'],
 'joy': ['tend', 'spontaneous'],
 'sadness': [],
 'shame': []}

In [167]:
save_obj(features, 'unigram_features')

In [170]:
save_obj(features_one_cat, 'unigram_features_one_cat')

In [ ]:
features=load_obj('unigram_features')

In [204]:
emotions=[emotion for emotion in features]
emotions

['joy', 'fear', 'anger', 'sadness', 'disgust', 'shame', 'guilt']

In [183]:
all_emotions_dfs=[]
for emotion in features:
    emotion_df=pd.DataFrame()
    em_series=pd.Series(features[emotion])
    words=pd.Series([item[0] for item in em_series])
    values=pd.Series([item[1] for item in em_series])
    emotion_df['words']=words
    emotion_df['values']=values
    all_emotions_dfs.append(emotion_df)

In [188]:
sorted_dfs=[]
for df in all_emotions_dfs:
    sorted_df=df.sort_values('values', ascending=False)
    sorted_dfs.append(sorted_df)

In [190]:
top_450=[df.iloc[:450] for df in sorted_dfs]

In [264]:
top_450

[             words    values
 294            joy  0.060860
 1458         happy  0.041214
 1563        passed  0.019923
 206       accepted  0.013350
 1279    university  0.013285
 66        selected  0.013011
 1287          exam  0.012144
 992          first  0.011935
 1537           got  0.011120
 1014          glad  0.010716
 860       received  0.010668
 1788          good  0.009688
 1410          year  0.009435
 1586      admitted  0.008094
 412    examination  0.007869
 571           long  0.007810
 784         result  0.007808
 413            met  0.007651
 91            love  0.007625
 76          letter  0.007187
 1082          born  0.006265
 869           seen  0.005743
 858        winning  0.005414
 427         school  0.005221
 1466      birthday  0.005148
 210        present  0.004874
 1047         birth  0.004701
 512           gave  0.004652
 670           form  0.004420
 342         joyful  0.004280
 ...            ...       ...
 1446       wanting  0.000324
 667      

In [246]:
final_features=[]
for index, df in enumerate(top_450):
    for i, row in df.iterrows():
        final_features.append(({'word':row[0]}, emotions[index]))

In [213]:
final_features

[({'word': 'joy'}, 'joy'),
 ({'word': 'happy'}, 'joy'),
 ({'word': 'passed'}, 'joy'),
 ({'word': 'accepted'}, 'joy'),
 ({'word': 'university'}, 'joy'),
 ({'word': 'selected'}, 'joy'),
 ({'word': 'exam'}, 'joy'),
 ({'word': 'first'}, 'joy'),
 ({'word': 'got'}, 'joy'),
 ({'word': 'glad'}, 'joy'),
 ({'word': 'received'}, 'joy'),
 ({'word': 'good'}, 'joy'),
 ({'word': 'year'}, 'joy'),
 ({'word': 'admitted'}, 'joy'),
 ({'word': 'examination'}, 'joy'),
 ({'word': 'long'}, 'joy'),
 ({'word': 'result'}, 'joy'),
 ({'word': 'met'}, 'joy'),
 ({'word': 'love'}, 'joy'),
 ({'word': 'letter'}, 'joy'),
 ({'word': 'born'}, 'joy'),
 ({'word': 'seen'}, 'joy'),
 ({'word': 'winning'}, 'joy'),
 ({'word': 'school'}, 'joy'),
 ({'word': 'birthday'}, 'joy'),
 ({'word': 'present'}, 'joy'),
 ({'word': 'birth'}, 'joy'),
 ({'word': 'gave'}, 'joy'),
 ({'word': 'form'}, 'joy'),
 ({'word': 'joyful'}, 'joy'),
 ({'word': 'last'}, 'joy'),
 ({'word': 'time'}, 'joy'),
 ({'word': 'final'}, 'joy'),
 ({'word': 'study'}, 'joy'

In [324]:
X=[]
y=[]
for index, df in enumerate(top_450):
    for i, row in df.iterrows():
        X.append(row[0])
        y.append(emotions[index])
X=np.array(X)
y=np.array(y)
le=LabelEncoder()
vectorizer = HashingVectorizer(n_features=10000, binary=True)
# encode document
vector = vectorizer.transform(X)
#vector = vectorizer.transform(input_text)
# summarize encoded vector
X_num=vector.todense()
y=le.fit_transform(y)
X=X.reshape(-1, 1)

In [325]:
X_train, X_test, y_train, y_test=train_test_split(X_num, y, test_size=.3, random_state=0)

In [247]:
shuffle(final_features)
size = int(len(final_features) * 0.1)
train_set, test_set = final_features[size:], final_features[:size]

In [249]:
classifier=NaiveBayesClassifier.train(train_set)

In [242]:
classifier.show_most_informative_features(10)

Most Informative Features
                    word = 'physical'        joy : disgus =      1.3 : 1.0
                    word = 'really'          joy : sadnes =      1.3 : 1.0
                    word = 'date'            joy : anger  =      1.3 : 1.0
                    word = 'thus'            joy : anger  =      1.3 : 1.0
                    word = 'too'             joy : sadnes =      1.3 : 1.0
                    word = 'position'        joy : disgus =      1.3 : 1.0
                    word = 'vacation'        joy : anger  =      1.3 : 1.0
                    word = 'daughter'        joy : shame  =      1.3 : 1.0
                    word = 'camp'            joy : disgus =      1.3 : 1.0
                    word = 'again'           joy : sadnes =      1.3 : 1.0


In [261]:
nltk.classify.accuracy(classifier, test_set)

0.0761904761904762

<h3>A shoddy accuracy for sure. Let's try sklearn's MNB</h3>

In [309]:
y_train

array([4, 1, 5, ..., 5, 6, 3], dtype=int64)

In [323]:
X_num[X_num>0]

matrix([[1., 1., 1., ..., 1., 1., 1.]])

In [326]:
clf=MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [327]:
clf.score(X_test, y_test)

0.08783068783068783

<h3>Still shitty accuracy, what's going on?</h3>